In [0]:
%run ./conf_notebook

#### READ AND PLOT THE DATA

In [0]:
df_train = ps.read_csv(train_3_csv_file_path)
df_train = df_train.drop("_c0", axis=1)
df_train = df_train.to_pandas()
df_train = df_train.reset_index(drop=True)

X_train_ = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)
y_train_ = y_categorical

In [0]:
X_train_

,dur,proto_encoded,service_encoded,state_encoded,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt
0,0.179469,-1.0,0.000000,0.0,0.4,0.2,-0.131902,0.007260,-0.025220,-0.010417,0.892857,-0.009738,0.253129,0.000000,0.0,0.435473,0.163928,0.012007,0.102883,1.0,0.324406,1.150945,1.0,0.000000
1,0.970450,-1.0,0.000000,0.0,1.2,3.6,0.233129,37.976407,-0.025185,-1.000000,0.884921,-0.009803,18.032918,0.666667,8.5,0.900023,0.302174,0.024441,12.068622,1.0,0.739772,1.608104,1.0,0.000000
2,2.427244,-1.0,0.000000,0.0,0.6,1.4,-0.050613,11.816697,-0.025700,-1.000000,0.884921,-0.009880,2.136200,0.333333,3.0,4.199465,2.012246,6.835484,99.320499,1.0,1.104087,1.548390,1.0,1.708847
3,2.514830,-1.0,0.141403,0.0,1.0,1.0,0.151840,0.549909,-0.025704,-1.000000,0.884921,-0.009867,0.068652,0.333333,1.5,2.766997,1.767374,0.103084,43.410362,1.0,0.577632,0.547346,1.0,0.000000
4,0.670406,-1.0,0.000000,0.0,0.8,0.4,0.079755,0.094374,-0.025546,0.000000,0.884921,-0.009801,0.091221,0.666667,0.5,0.860772,1.481864,0.961223,1.007099,1.0,1.271038,1.033172,1.0,1.960584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,-0.002355,0.0,1.000000,1.0,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,0.000000,0.0,-0.004909,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
175337,0.754692,-1.0,0.000000,0.0,0.8,0.6,0.145706,0.172414,-0.025544,0.000000,0.884921,-0.009798,0.124133,0.666667,0.5,0.981350,1.311864,1.480554,1.045109,1.0,1.835898,1.804430,1.0,1.518608
175338,-0.002355,0.0,1.000000,1.0,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,0.000000,0.0,-0.004909,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
175339,-0.002355,0.0,1.000000,1.0,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,0.000000,0.0,-0.004909,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [0]:
df_test = ps.read_csv(test_3_csv_file_path)
df_test = df_test.drop("_c0", axis=1)
df_test = df_test.to_pandas()
df_test = df_test.reset_index(drop=True)

X_test_ = df_test.drop('attack_cat', axis=1)  
y = df_test['attack_cat']

y_encoded = label_encoder.transform(y)
y_categorical = to_categorical(y_encoded)
y_test_ = y_categorical

In [0]:
X_test_

,dur,proto_encoded,service_encoded,state_encoded,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt
0,-0.002352,0.000000,0.0,1.0,0.0,-0.2,0.050613,-0.148820,0.701650,0.000000,-0.115079,2.019491,-0.051967,0.000000,0.0,-0.004873,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,-0.002356,0.000000,0.0,1.0,0.0,-0.2,1.021472,-0.148820,0.974449,0.000000,-0.115079,9.902808,-0.051967,0.000000,0.0,-0.004927,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,-0.002361,0.000000,0.0,1.0,0.0,-0.2,0.489264,-0.148820,1.574607,0.000000,-0.115079,9.603514,-0.051967,0.000000,0.0,-0.004982,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,-0.002359,0.000000,0.0,1.0,0.0,-0.2,0.360429,-0.148820,1.307870,0.000000,-0.115079,6.741094,-0.051967,0.000000,0.0,-0.004964,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,-0.002353,0.000000,0.0,1.0,0.0,-0.2,1.300613,-0.148820,0.774397,0.000000,-0.115079,9.558507,-0.051967,0.000000,0.0,-0.004891,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,-0.002361,0.000000,0.0,1.0,0.0,-0.2,-0.250000,-0.148820,1.574607,0.000000,-0.115079,0.926240,-0.051967,0.000000,0.0,-0.004982,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
82328,1.653321,-1.000000,0.0,0.0,1.8,0.6,13.521472,0.172414,-0.025618,0.000000,0.884921,-0.008501,0.028554,2.333333,0.5,1.008186,2.814604,1.909100,1.660838,1.0,0.559588,1.716222,1.0,2.645164
82329,-0.002368,-2.895909,0.0,1.0,-0.1,-0.2,-0.294479,-0.148820,-0.025813,-1.322917,-0.115079,-0.009898,-0.051967,0.000000,0.0,1087.971739,-0.000118,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
82330,-0.002368,-2.895909,0.0,1.0,-0.1,-0.2,-0.294479,-0.148820,-0.025813,-1.322917,-0.115079,-0.009898,-0.051967,0.000000,0.0,1087.971956,-0.000118,0.004359,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [0]:
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# Print the mapping
print(label_mapping)

{'Analysis': 0, 'Backdoor': 1, 'DoS': 2, 'Exploits': 3, 'Fuzzers': 4, 'Generic': 5, 'Normal': 6, 'Reconnaissance': 7, 'Shellcode': 8, 'Worms': 9}


In [0]:
# # Define training data
# train_partition = 4

# train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
# df_train = spark.read.parquet(train_path)
# df_train = ps.DataFrame(df_train).to_pandas()
# df_train = df_train.reset_index(drop=True)

# print(df_train["attack_cat"].value_counts())

# train_counts = df_train['attack_cat'].value_counts()

# # Define the desired order of the class labels
# desired_order = ['Normal', 'Generic', 'Exploits', 'Fuzzers', 'DoS', 'Reconnaissance', 'Analysis', 'Backdoor', 'Shellcode', 'Worms']

# # Reindex the train_counts Series to match the desired order
# train_counts = train_counts.reindex(desired_order)

# # Create an interactive bar plot for the training data with steel blue color and thinner bars
# fig = px.bar(train_counts, x=train_counts.index, y=train_counts.values, labels={'x': 'Attack Category', 'y': 'Number of records'},
#              color_discrete_sequence=['steelblue'])

# # Update the bar width (e.g., 0.4 for thinner bars)
# fig.update_traces(width=0.4)

# fig.update_layout(
#     title=f'Training Data {train_partition}. Total number of records = {df_train.shape[0]}',
#     height=500,  # Set the height of the graph
#     width=800    # Set the width of the graph
# )

# # Show the plot
# fig.show()

#### FUNCTIONS

In [0]:
def train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[]):

  # BASELINE ANN --------------------------------------------------------------------------------------------

  early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    verbose=1,           
    restore_best_weights=True  
  )

  model = Sequential()
  model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  history = model.fit(
      X_train, y_train,
      validation_data=(X_test, y_test),
      epochs=50,
      batch_size=12,
      callbacks=[early_stopping] 
  )
  
  class_labels = label_encoder.classes_
  all_class_indices = range(len(class_labels))
  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Test accuracy: {accuracy*100:.2f}%')

  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_true_classes = np.argmax(y_test, axis=1)

  report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
  print(report)

  # FINE-TUNE ANN --------------------------------------------------------------------------------------------

  df_train = ps.read_csv(train_3_csv_file_path).to_pandas()
  df_train = df_train.drop("_c0", axis=1)
  df_train = df_train.reset_index(drop=True)
  df_train = df_train[~df_train["attack_cat"].isin(exclude_categories)]

  X = df_train.drop('attack_cat', axis=1)  
  y = df_train['attack_cat']

  y_encoded = label_encoder.transform(y)
  print(np.unique(y_encoded))
  y_categorical = to_categorical(y_encoded)
  print(y_categorical.shape)

  X_train, X_test, y_train, y_test = train_test_split(
      X, y_categorical, test_size=0.2, random_state=42, stratify=y
  )

  history = model.fit(
      X_train, y_train,
      validation_data=(X_test, y_test),
      epochs=50,
      batch_size=12,
      callbacks=[early_stopping]  
  )

  class_labels = label_encoder.classes_
  all_class_indices = range(len(class_labels))

  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Test accuracy: {accuracy*100:.2f}%')

  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_true_classes = np.argmax(y_test, axis=1)

  report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
  print(report)

  return model

In [0]:
def train_base_ann_and_fine_tune_final(X_train, y_train, y_categorical, epochs_base, epochs_fined, exclude_categories=[]):

  # BASELINE ANN --------------------------------------------------------------------------------------------

  model = Sequential()
  model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  history = model.fit(
      X_train, y_train,
      epochs=epochs_base,
      batch_size=12
  )

  # FINE-TUNE ANN --------------------------------------------------------------------------------------------

  df_train = ps.read_csv(train_3_csv_file_path).to_pandas()
  df_train = df_train.drop("_c0", axis=1)
  df_train = df_train.reset_index(drop=True)
  df_train = df_train[~df_train["attack_cat"].isin(exclude_categories)]

  X = df_train.drop('attack_cat', axis=1)  
  y = df_train['attack_cat']

  y_encoded = label_encoder.transform(y)
  print(np.unique(y_encoded))
  y_categorical = to_categorical(y_encoded)
  print(y_categorical.shape)

  X_train, y_train, = X, y_categorical

  history = model.fit(
      X_train, y_train,
      epochs=epochs_fined,
      batch_size=12
  )

  return model

#### ANN 0

In [0]:
train_partition = 0

train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])


[0 1 2 3 4 5 6 7 8 9]
(37421, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2495/2495 ━━━━━━━━━━━━━━━━━━━━ 3s 903us/step - accuracy: 0.4401 - loss: 1.7222 - val_accuracy: 0.5440 - val_loss: 1.3131
Epoch 2/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 858us/step - accuracy: 0.5433 - loss: 1.1962 - val_accuracy: 0.5555 - val_loss: 1.1389
Epoch 3/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 849us/step - accuracy: 0.5548 - loss: 1.0965 - val_accuracy: 0.5631 - val_loss: 1.0805
Epoch 4/50
1577/2495 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.5688 - l

*** WARNING: max output size exceeded, skipping output. ***

2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 853us/step - accuracy: 0.6442 - loss: 0.8276 - val_accuracy: 0.6395 - val_loss: 0.8728
Epoch 41/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 866us/step - accuracy: 0.6479 - loss: 0.8159 - val_accuracy: 0.6186 - val_loss: 0.9280
Epoch 42/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 856us/step - accuracy: 0.6393 - loss: 0.8285 - val_accuracy: 0.6314 - val_loss: 0.8807
Epoch 43/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 852us/step - accuracy: 0.6447 - loss: 0.8

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 9033/11690 ━━━━━━━━━━━━━━━━━━━━ 1s 7

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 855us/step - accuracy: 0.7997 - loss: 0.5054 - val_accuracy: 0.8005 - val_loss: 0.5182
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 15.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - accuracy: 0.8001 - loss: 0.5020
Test accuracy: 80.14%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step
                precision    recall  f1-score   support

      Analysis     1.0000    0.0025    0.0050       400
      Backdoor     0.7714    0.0774    0.1406       349
           DoS     0.6260    0.0314    0.0598      2453
      Exploits     0.5968    0.9295    0.7269      6679
       Fuzzers     0.6412    0.6637    0.6523      3637
       Generic     0.9981    0.9790    0.9885      8000
        Normal     0.9110    0.8905    0.9007     11200
Reconnaissance     0.8387    0.7312    0.7813     

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3119/3119 ━━━━━━━━━━━━━━━━━━━━ 3s 724us/step - accuracy: 0.4482 - loss: 1.5999
Epoch 2/50
3119/3119 ━━━━━━━━━━━━━━━━━━━━ 2s 715us/step - accuracy: 0.5463 - loss: 1.1456
Epoch 3/50
2763/3119 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.5572 - loss: 1.0895

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 11s 752us/step - accuracy: 0.8060 - loss: 0.4896


In [0]:

y_pred_0 = model.predict(X_train_)

soft_classification_0 = pd.DataFrame(y_pred_0, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_0['ANN'] = 0
soft_classification_0 = soft_classification_0.reset_index()

soft_classification_0.head()

y_pred_classes = np.argmax(y_pred_0, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 617us/step
                precision    recall  f1-score   support

      Analysis     1.0000    0.0020    0.0040      2000
      Backdoor     0.8268    0.0848    0.1538      1746
           DoS     0.6269    0.0377    0.0711     12264
      Exploits     0.6045    0.9320    0.7333     33393
       Fuzzers     0.6603    0.6392    0.6496     18184
       Generic     0.9990    0.9784    0.9886     40000
        Normal     0.8955    0.9114    0.9034     56000
Reconnaissance     0.9110    0.6935    0.7875     10491
     Shellcode     0.3771    0.3928    0.3848      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.8056    175341
     macro avg     0.6901    0.4672    0.4676    175341
  weighted avg     0.8179    0.8056    0.7772    175341



In [0]:
y_pred_0_test = model.predict(X_test_)

soft_classification_0_test = pd.DataFrame(y_pred_0_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_0_test['ANN'] = 0
soft_classification_0_test = soft_classification_0_test.reset_index()

soft_classification_0_test.head()

y_pred_classes = np.argmax(y_pred_0_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 618us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.4464    0.0429    0.0782       583
           DoS     0.4677    0.0514    0.0926      4089
      Exploits     0.5602    0.9058    0.6923     11132
       Fuzzers     0.3192    0.5049    0.3911      6062
       Generic     0.9989    0.9623    0.9803     18871
        Normal     0.9373    0.8166    0.8728     37000
Reconnaissance     0.8235    0.7486    0.7842      3496
     Shellcode     0.2274    0.3862    0.2863       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7836     82332
     macro avg     0.4781    0.4419    0.4178     82332
  weighted avg     0.8118    0.7836    0.7791     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 4

In [0]:
train_partition = 4

train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[0 1 2 3 4 5 6 7 9]
(17092, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1140/1140 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7149 - loss: 1.6380 - val_accuracy: 0.7912 - val_loss: 0.8915
Epoch 2/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 900us/step - accuracy: 0.8009 - loss: 0.7878 - val_accuracy: 0.8195 - val_loss: 0.8184
Epoch 3/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step - accuracy: 0.8035 - loss: 0.6619 - val_accuracy: 0.8154 - val_loss: 0.5738
Epoch 4/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - accuracy: 0.8153 - loss: 0.5888 - val_accuracy: 0.8219 - val_loss: 0.6083
Epoch 5/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.8193 - loss: 0.5787 - val_accuracy: 0.8210 - val_loss: 0.5548
Epoch 6/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 936us/step - accuracy: 0.8180 - loss: 0.5642 - val_accuracy: 0.8052 - val_loss: 0.5877
Epoch 7/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 1s 905us/step - accuracy: 0.8093 - loss: 0.5769 - val_accuracy: 0.8257 - val_loss: 0.5590
Epoch 8/50
 198/1140 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.8150 - loss: 0.5220

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8647/11690 ━━━━━━━━━━━━━━━━━━━━ 2s 7

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 11s 899us/step - accuracy: 0.7914 - loss: 0.5337 - val_accuracy: 0.7925 - val_loss: 0.5303
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 605us/step - accuracy: 0.7821 - loss: 0.5662
Test accuracy: 78.19%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 618us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       400
      Backdoor     1.0000    0.0029    0.0057       349
           DoS     0.3194    0.0375    0.0671      2453
      Exploits     0.5831    0.9132    0.7117      6679
       Fuzzers     0.5934    0.5876    0.5905      3637
       Generic     0.9966    0.9786    0.9875      8000
        Normal     0.8900    0.8874    0.8887     11200
Reconnaissance     0.7914    0.6258    0.6990      2

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
1425/1425 ━━━━━━━━━━━━━━━━━━━━ 2s 734us/step - accuracy: 0.7530 - loss: 1.9140
Epoch 2/50
1425/1425 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step - accuracy: 0.7984 - loss: 0.7910
Epoch 3/50
1425/1425 ━━━━━━━━━━━━━━━━━━━━ 1s 741us/step - accuracy: 0.8114 - loss: 0.6732
Epoch 4/50
1425/1425 ━━━━━━━━━━━━━━━━━━━━ 1s 728us/step - accuracy: 0.8147 - loss: 0.5944
Epoch 5/50
1425/1425 ━━━━━━━━━━━━━━━━━━━━ 1s 738us/step - accuracy: 0.8145 - loss: 0.5697
Epoch 6/50
1425/1425

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 11s 739us/step - accuracy: 0.8022 - loss: 0.5026


In [0]:
y_pred_1 = model.predict(X_train_)

soft_classification_1 = pd.DataFrame(y_pred_1, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_1['ANN'] = 4
soft_classification_1 = soft_classification_1.reset_index()

soft_classification_1.head()

y_pred_classes = np.argmax(y_pred_1, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 620us/step
                precision    recall  f1-score   support

      Analysis     0.8571    0.0030    0.0060      2000
      Backdoor     0.6357    0.0939    0.1637      1746
           DoS     0.6154    0.0404    0.0759     12264
      Exploits     0.6080    0.8945    0.7240     33393
       Fuzzers     0.5979    0.7551    0.6674     18184
       Generic     0.9966    0.9791    0.9878     40000
        Normal     0.9343    0.8663    0.8990     56000
Reconnaissance     0.7704    0.8020    0.7859     10491
     Shellcode     0.7222    0.0229    0.0445      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.8007    175341
     macro avg     0.6738    0.4457    0.4354    175341
  weighted avg     0.8135    0.8007    0.7739    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_1_test = model.predict(X_test_)

soft_classification_1_test = pd.DataFrame(y_pred_1_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_1_test['ANN'] = 4
soft_classification_1_test = soft_classification_1_test.reset_index()

soft_classification_1_test.head()

y_pred_classes = np.argmax(y_pred_1_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 612us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.2348    0.0463    0.0774       583
           DoS     0.4575    0.0514    0.0923      4089
      Exploits     0.5575    0.8708    0.6798     11132
       Fuzzers     0.2764    0.6160    0.3816      6062
       Generic     0.9960    0.9644    0.9799     18871
        Normal     0.9607    0.7302    0.8297     37000
Reconnaissance     0.6649    0.8418    0.7430      3496
     Shellcode     0.3947    0.0397    0.0721       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7511     82332
     macro avg     0.4543    0.4161    0.3856     82332
  weighted avg     0.8102    0.7511    0.7545     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 2

In [0]:
train_partition = 2

train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=[])

[2 3 4 5 6 7 8 9]
(47671, 10)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3178/3178 ━━━━━━━━━━━━━━━━━━━━ 4s 908us/step - accuracy: 0.8689 - loss: 0.4357 - val_accuracy: 0.9512 - val_loss: 0.1392
Epoch 2/50
3178/3178 ━━━━━━━━━━━━━━━━━━━━ 3s 870us/step - accuracy: 0.9532 - loss: 0.1295 - val_accuracy: 0.9546 - val_loss: 0.1236
Epoch 3/50
2576/3178 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accur

*** WARNING: max output size exceeded, skipping output. ***

3178/3178 ━━━━━━━━━━━━━━━━━━━━ 3s 891us/step - accuracy: 0.9596 - loss: 0.1004 - val_accuracy: 0.9631 - val_loss: 0.1039
Epoch 19/50
3178/3178 ━━━━━━━━━━━━━━━━━━━━ 5s 880us/step - accuracy: 0.9622 - loss: 0.0936 - val_accuracy: 0.9632 - val_loss: 0.1021
Epoch 20/50
3178/3178 ━━━━━━━━━━━━━━━━━━━━ 3s 924us/step - accuracy: 0.9594 - loss: 0.0988 - val_accuracy: 0.9626 - val_loss: 0.1067
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 15.
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - accuracy: 0.9600 - loss: 0.1008
Test accuracy: 95.98%
298/298 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
       

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7 8 9]
(175341, 10)
Epoch 1/50
 8701/11690 ━━━━━━━━━━━━━

*** WARNING: max output size exceeded, skipping output. ***

11690/11690 ━━━━━━━━━━━━━━━━━━━━ 10s 887us/step - accuracy: 0.7813 - loss: 0.5617 - val_accuracy: 0.7787 - val_loss: 0.5855
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 611us/step - accuracy: 0.7492 - loss: 0.6514
Test accuracy: 74.85%
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 624us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       400
      Backdoor     0.0000    0.0000    0.0000       349
           DoS     0.2902    0.0265    0.0486      2453
      Exploits     0.5323    0.8852    0.6648      6679
       Fuzzers     0.5096    0.7740    0.6146      3637
       Generic     0.9989    0.9786    0.9886      8000
        Normal     0.9533    0.7403    0.8334     11200
Reconnaissance     0.8293    0.6206    0.7099      2098
     She

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=[])

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
3973/3973 ━━━━━━━━━━━━━━━━━━━━ 4s 727us/step - accuracy: 0.9054 - loss: 0.3501
Epoch 2/50
3973/3973 ━━━━━━━━━━━━━━━━━━━━ 3s 741us/step - accuracy: 0.9537 - loss: 0.1194
Epoch 3/50
 824/3973 ━━━━━━━━━━━━━━━━━━━━ 2s 734us/step - accuracy: 0.9508 - loss: 0.1176

*** WARNING: max output size exceeded, skipping output. ***

14612/14612 ━━━━━━━━━━━━━━━━━━━━ 11s 740us/step - accuracy: 0.7971 - loss: 0.5253


In [0]:
y_pred_2 = model.predict(X_train_)

soft_classification_2 = pd.DataFrame(y_pred_2, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_2['ANN'] = 2
soft_classification_2 = soft_classification_2.reset_index()

soft_classification_2.head()

y_pred_classes = np.argmax(y_pred_2, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 602us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.8643    0.0693    0.1283      1746
           DoS     0.6696    0.0126    0.0247     12264
      Exploits     0.5940    0.9350    0.7265     33393
       Fuzzers     0.5744    0.8113    0.6726     18184
       Generic     0.9990    0.9778    0.9883     40000
        Normal     0.9656    0.8238    0.8891     56000
Reconnaissance     0.8230    0.7518    0.7858     10491
     Shellcode     0.6068    0.1103    0.1867      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7957    175341
     macro avg     0.6097    0.4492    0.4402    175341
  weighted avg     0.8176    0.7957    0.7687    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_2_test = model.predict(X_test_)

soft_classification_2_test = pd.DataFrame(y_pred_2_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7', 'class_prob_8', 'class_prob_9'])
soft_classification_2_test['ANN'] = 2
soft_classification_2_test = soft_classification_2_test.reset_index()

soft_classification_2_test.head()

y_pred_classes = np.argmax(y_pred_2_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 593us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.3571    0.0343    0.0626       583
           DoS     0.4899    0.0179    0.0345      4089
      Exploits     0.5365    0.9242    0.6789     11132
       Fuzzers     0.2482    0.6617    0.3610      6062
       Generic     0.9991    0.9621    0.9802     18871
        Normal     0.9742    0.6442    0.7755     37000
Reconnaissance     0.6981    0.8029    0.7468      3496
     Shellcode     0.3481    0.1243    0.1832       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7195     82332
     macro avg     0.4651    0.4171    0.3823     82332
  weighted avg     0.8157    0.7195    0.7263     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 3

In [0]:
train_partition = 3

train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=["Shellcode", "Worms"])

[0 1 2 3 4 5 6 7]
(44588, 8)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2973/2973 ━━━━━━━━━━━━━━━━━━━━ 4s 992us/step - accuracy: 0.9894 - loss: 0.0522 - val_accuracy: 0.9926 - val_loss: 0.0373
Epoch 2/50
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 3s 885us/step - accuracy: 0.9932 - loss: 0.0284 - val_accuracy: 0.9922 - val_loss: 0.0227
Epoch 3/50
2801/2973 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/

*** WARNING: max output size exceeded, skipping output. ***

2973/2973 ━━━━━━━━━━━━━━━━━━━━ 3s 876us/step - accuracy: 0.9940 - loss: 0.0179 - val_accuracy: 0.9930 - val_loss: 0.0190
Epoch 13/50
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 3s 887us/step - accuracy: 0.9931 - loss: 0.0191 - val_accuracy: 0.9933 - val_loss: 0.0194
Epoch 14/50
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 3s 883us/step - accuracy: 0.9937 - loss: 0.0304 - val_accuracy: 0.9932 - val_loss: 0.0194
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.9937 - loss: 0.0148
Test accuracy: 99.32%
279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
                prec

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7]
(174078, 8)
Epoch 1/50
 8870/11606 ━━━━━━━━━━━━━━━━━━━━ 1s 727

*** WARNING: max output size exceeded, skipping output. ***

11606/11606 ━━━━━━━━━━━━━━━━━━━━ 10s 880us/step - accuracy: 0.7967 - loss: 0.5137 - val_accuracy: 0.7985 - val_loss: 0.5162
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 596us/step - accuracy: 0.7790 - loss: 0.5745
Test accuracy: 77.86%
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 609us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       400
      Backdoor     0.0000    0.0000    0.0000       349
           DoS     0.3205    0.0102    0.0198      2453
      Exploits     0.5682    0.9043    0.6979      6679
       Fuzzers     0.5581    0.7014    0.6216      3637
       Generic     0.9959    0.9786    0.9872      8000
        Normal     0.9360    0.8303    0.8800     11200
Reconnaissance     0.7840    0.6506    0.7111      2098

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=["Shellcode", "Worms"])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



3716/3716 ━━━━━━━━━━━━━━━━━━━━ 3s 722us/step - accuracy: 0.9718 - loss: 0.5904
Epoch 2/50
3716/3716 ━━━━━━━━━━━━━━━━━━━━ 3s 731us/step - accuracy: 0.9934 - loss: 0.0233
Epoch 3/50
1429/3716 ━━━━━━━━━━━━━━━━━━━━ 1s 742us/step - accuracy: 0.9931 - loss: 0.0193

*** WARNING: max output size exceeded, skipping output. ***

14507/14507 ━━━━━━━━━━━━━━━━━━━━ 11s 735us/step - accuracy: 0.7995 - loss: 0.5058


In [0]:
y_pred_3 = model.predict(X_train_)

soft_classification_3 = pd.DataFrame(y_pred_3, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_3['class_prob_8'] = 0
soft_classification_3['class_prob_9'] = 0
soft_classification_3['ANN'] = 3
soft_classification_3 = soft_classification_3.reset_index()

y_pred_classes = np.argmax(y_pred_3, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 620us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.7379    0.0613    0.1132      1746
           DoS     0.7832    0.0274    0.0529     12264
      Exploits     0.6066    0.9034    0.7258     33393
       Fuzzers     0.6069    0.6962    0.6485     18184
       Generic     0.9975    0.9792    0.9883     40000
        Normal     0.8996    0.8852    0.8923     56000
Reconnaissance     0.8231    0.7690    0.7952     10491
     Shellcode     0.0000    0.0000    0.0000      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7989    175341
     macro avg     0.5455    0.4322    0.4216    175341
  weighted avg     0.8047    0.7989    0.7683    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_3_test = model.predict(X_test_)

soft_classification_3_test = pd.DataFrame(y_pred_3_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_3_test['class_prob_8'] = 0
soft_classification_3_test['class_prob_9'] = 0
soft_classification_3_test['ANN'] = 3
soft_classification_3_test = soft_classification_3_test.reset_index()

soft_classification_3_test.head()

y_pred_classes = np.argmax(y_pred_3_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 609us/step
                precision    recall  f1-score   support

      Analysis     0.3500    0.0103    0.0201       677
      Backdoor     0.4255    0.0343    0.0635       583
           DoS     0.5543    0.0362    0.0680      4089
      Exploits     0.5615    0.8864    0.6875     11132
       Fuzzers     0.2862    0.5541    0.3775      6062
       Generic     0.9978    0.9636    0.9804     18871
        Normal     0.9402    0.7757    0.8501     37000
Reconnaissance     0.7226    0.8135    0.7653      3496
     Shellcode     0.0000    0.0000    0.0000       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7668     82332
     macro avg     0.4838    0.4074    0.3812     82332
  weighted avg     0.8123    0.7668    0.7640     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### ANN 1

In [0]:
# Define training data
train_partition = 1

train_path = f"{train_3_partitioned_parquet_file_path}/clustering_pred={train_partition}"
df_train = spark.read.parquet(train_path)
df_train = ps.DataFrame(df_train).to_pandas()
df_train = df_train.reset_index(drop=True)

X = df_train.drop('attack_cat', axis=1)  
y = df_train['attack_cat']

y_encoded = label_encoder.transform(y)
print(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded)
print(y_categorical.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

model = train_base_ann_and_fine_tune(X_train, X_test, y_train, y_test, y_categorical, exclude_categories=["Shellcode", "Worms"])

[0 1 2 3 4 5 6 7]
(28569, 8)
Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1905/1905 ━━━━━━━━━━━━━━━━━━━━ 3s 959us/step - accuracy: 0.4333 - loss: 1.6492 - val_accuracy: 0.4447 - val_loss: 1.5575
Epoch 2/50
1905/1905 ━━━━━━━━━━━━━━━━━━━━ 2s 900us/step - accuracy: 0.4476 - loss: 1.4338 - val_accuracy: 0.4470 - val_loss: 1.5257
Epoch 3/50
1905/1905 ━━━━━━━━━━━━━━━━━━━━ 3s 897us/step - accuracy: 0.4484 - loss: 1.4188 - val_accuracy: 0.4491 - val_loss: 1.4128
Epoch 4/50
1905/1905 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.4562 - loss: 1.3872 - val_accuracy: 0.4515 - val_loss: 1.4624
Epoch 5/50
 811/1905 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.4469 - loss: 1.4310

*** WARNING: max output size exceeded, skipping output. ***

1905/1905 ━━━━━━━━━━━━━━━━━━━━ 2s 914us/step - accuracy: 0.4459 - loss: 1.4032 - val_accuracy: 0.4461 - val_loss: 1.4167
Epoch 13/50
1905/1905 ━━━━━━━━━━━━━━━━━━━━ 2s 906us/step - accuracy: 0.4514 - loss: 1.3839 - val_accuracy: 0.4489 - val_loss: 1.3891
Epoch 14/50
1905/1905 ━━━━━━━━━━━━━━━━━━━━ 

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

[0 1 2 3 4 5 6 7]
(174078, 8)
Epoch 1/50
 8561/11606 ━━━━━━━━━━━━━━━━━━━━ 2s 753us

*** WARNING: max output size exceeded, skipping output. ***

11606/11606 ━━━━━━━━━━━━━━━━━━━━ 10s 886us/step - accuracy: 0.8069 - loss: 0.4801 - val_accuracy: 0.8078 - val_loss: 0.4894
Epoch 30: early stopping
Restoring model weights from the end of the best epoch: 25.
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 0.8085 - loss: 0.4826
Test accuracy: 80.96%
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 1s 598us/step
                precision    recall  f1-score   support

      Analysis     1.0000    0.0025    0.0050       400
      Backdoor     0.6182    0.0974    0.1683       349
           DoS     0.5882    0.0285    0.0544      2453
      Exploits     0.6038    0.9277    0.7315      6679
       Fuzzers     0.6525    0.6970    0.6740      3637
       Generic     0.9986    0.9781    0.9883      8000
        Normal     0.9165    0.8935    0.9048     11200
Reconnaissance     0.8736    0.7245    0.7921      

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `

In [0]:
model = train_base_ann_and_fine_tune_final(X, y_categorical, y_categorical, epochs_base=50, epochs_fined=50, exclude_categories=["Shellcode", "Worms"])

Epoch 1/50


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2381/2381 ━━━━━━━━━━━━━━━━━━━━ 3s 736us/step - accuracy: 0.4316 - loss: 1.7259
Epoch 2/50
2381/2381 ━━━━━━━━━━━━━━━━━━━━ 2s 742us/step - accuracy: 0.4459 - loss: 1.4026
Epoch 3/50
2381/2381 ━━━━━━━━━━━━━━━━━━━━ 2s 754us/step - accuracy: 0.4490 - loss: 1.3927
Epoch 4/50
1580/2381 ━━━━━━━━━━━━━━━━

*** WARNING: max output size exceeded, skipping output. ***

14507/14507 ━━━━━━━━━━━━━━━━━━━━ 20s 752us/step - accuracy: 0.8020 - loss: 0.5047


In [0]:
y_pred_4 = model.predict(X_train_)

soft_classification_4 = pd.DataFrame(y_pred_4, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_4['class_prob_8'] = 0
soft_classification_4['class_prob_9'] = 0
soft_classification_4['ANN'] = 1
soft_classification_4 = soft_classification_4.reset_index()

y_pred_classes = np.argmax(y_pred_4, axis=1)
y_true_classes = np.argmax(y_train_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels)) 
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

5480/5480 ━━━━━━━━━━━━━━━━━━━━ 3s 623us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000      2000
      Backdoor     0.7041    0.0790    0.1421      1746
           DoS     0.4767    0.0217    0.0415     12264
      Exploits     0.5948    0.9172    0.7216     33393
       Fuzzers     0.5889    0.7385    0.6553     18184
       Generic     0.9993    0.9779    0.9885     40000
        Normal     0.9286    0.8630    0.8946     56000
Reconnaissance     0.8699    0.7548    0.8083     10491
     Shellcode     0.0000    0.0000    0.0000      1133
         Worms     0.0000    0.0000    0.0000       130

      accuracy                         0.7974    175341
     macro avg     0.5162    0.4352    0.4252    175341
  weighted avg     0.7913    0.7974    0.7693    175341



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [0]:
y_pred_4_test = model.predict(X_test_)

soft_classification_4_test = pd.DataFrame(y_pred_4_test, columns=['class_prob_0', 'class_prob_1', 'class_prob_2', 'class_prob_3', 'class_prob_4', 'class_prob_5', 'class_prob_6', 'class_prob_7'])
soft_classification_4_test['class_prob_8'] = 0
soft_classification_4_test['class_prob_9'] = 0
soft_classification_4_test['ANN'] = 1
soft_classification_4_test = soft_classification_4_test.reset_index()

y_pred_classes = np.argmax(y_pred_4_test, axis=1)
y_true_classes = np.argmax(y_test_ , axis=1)  
class_labels = label_encoder.classes_
all_class_indices = range(len(class_labels))
report = classification_report(y_true_classes, y_pred_classes, labels=all_class_indices, target_names=class_labels, digits=4)
print(report)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 2s 623us/step
                precision    recall  f1-score   support

      Analysis     0.0000    0.0000    0.0000       677
      Backdoor     0.3750    0.0463    0.0824       583
           DoS     0.4131    0.0262    0.0492      4089
      Exploits     0.5381    0.8980    0.6729     11132
       Fuzzers     0.2631    0.5947    0.3648      6062
       Generic     0.9993    0.9617    0.9801     18871
        Normal     0.9565    0.7285    0.8271     37000
Reconnaissance     0.8282    0.8001    0.8139      3496
     Shellcode     0.0000    0.0000    0.0000       378
         Worms     0.0000    0.0000    0.0000        44

      accuracy                         0.7486     82332
     macro avg     0.4373    0.4055    0.3791     82332
  weighted avg     0.8094    0.7486    0.7518     82332



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



path,name,size,modificationTime
dbfs:/mnt/pqsashrd/data/ann_pred_1.csv/,ann_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_1_test.csv/,ann_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_3.csv/,ann_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1.csv/,clust_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1_test.csv/,clust_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3.csv/,clust_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3_test.csv/,clust_pred_3_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/test_1.csv,test_1.csv,38574669,1714569841000
dbfs:/mnt/pqsashrd/data/test_2.csv,test_2.csv,35056701,1714569835000
dbfs:/mnt/pqsashrd/data/test_3.csv,test_3.csv,25575183,1714569796000


#### SAVE PREDICTIONS

In [0]:
dataframes = [soft_classification_0, soft_classification_1, soft_classification_2, soft_classification_3, soft_classification_4]
soft_classification = pd.concat(dataframes, axis=0)

In [0]:
soft_classification[soft_classification["index"]==0]

,index,class_prob_0,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,ANN
0,0,1.571106e-28,0.000008,4.447967e-01,3.685432e-02,8.785637e-02,1.719398e-03,0.428729,3.648511e-05,3.391861e-08,4.706974e-32,0
0,0,0.000000e+00,0.004884,9.021448e-04,6.328616e-05,3.668739e-03,3.818160e-07,0.990474,7.271972e-06,6.978870e-08,2.977949e-23,4
0,0,0.000000e+00,0.000000,5.645400e-18,1.642232e-14,1.077427e-11,0.000000e+00,1.000000,0.000000e+00,6.250002e-35,0.000000e+00,2
0,0,9.381504e-34,0.000823,2.951417e-03,4.523185e-04,2.170290e-01,4.040514e-04,0.778340,2.157764e-08,0.000000e+00,0.000000e+00,3
0,0,1.169223e-27,0.000020,1.904441e-03,2.235340e-03,9.355816e-02,3.084503e-04,0.896326,5.647199e-03,0.000000e+00,0.000000e+00,1


In [0]:
soft_classification = ps.DataFrame(soft_classification).to_spark()
save_path = ann_pred_3_csv_file_path
soft_classification.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)

In [0]:
dataframes = [soft_classification_0_test, soft_classification_1_test, soft_classification_2_test, soft_classification_3_test, soft_classification_4_test]
soft_classification_test = pd.concat(dataframes, axis=0)

In [0]:
soft_classification_test[soft_classification_test["index"]==0]

,index,class_prob_0,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,ANN
0,0,0.000000e+00,3.744149e-08,0.013977,0.036043,0.338816,0.001401,0.165077,3.197112e-04,0.444366,1.009547e-24,0
0,0,4.896259e-21,8.942207e-27,0.029868,0.036598,0.494221,0.001117,0.177211,2.401974e-05,0.260961,4.057520e-08,4
0,0,1.531922e-28,2.187836e-18,0.023698,0.028346,0.485866,0.003116,0.226898,2.912662e-05,0.231988,6.036138e-05,2
0,0,8.279034e-27,1.711027e-03,0.034384,0.055022,0.593307,0.016282,0.299295,5.022725e-08,0.000000,0.000000e+00,3
0,0,3.999977e-16,1.713951e-04,0.020492,0.060336,0.538342,0.018175,0.362476,7.122481e-06,0.000000,0.000000e+00,1


In [0]:
soft_classification_test = ps.DataFrame(soft_classification_test).to_spark()
save_path = ann_pred_3_test_csv_file_path
soft_classification_test.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)